# Background
This document describes how to do the variant filtering of a VCF format file generated using BCFTools [[1]](https://samtools.github.io/bcftools/). The approach followed consists on using a supervised machine learning method for the filtering, more specifically we will use different binary classifiers and we will assess the performance on a test dataset in order to decide which is superior.

# Data used
In this document we have used a VCF file generated for sample NA12878 using the sequencing data generated for the 1000 Genomes Project. The callset in the VCF file was generated using BCFTools.

# Traning the model
We are going to use the sites in our callset for chr20 that are also found by GIAB NA12878 [[2]](https://github.com/genome-in-a-bottle). GIAB sequenced NA12878 using 13 different sequencing technologies and analysis methods, so the GIAB callset is considered as the gold standard callset and this is why we considered the variants identified by GIAB and us as true sites that are useful to train the model. We will train the classifier independently for the SNPs and the INDELs

## Annotations used for the filtering exercise
BCFTools annotates each of the identified variants with a set of features used as predictors for our model. The variant annotations used are:


##INFO=<ID=IDV,Number=1,Type=Integer,Description="Maximum number of reads supporting an indel">
##INFO=<ID=IMF,Number=1,Type=Float,Description="Maximum fraction of reads supporting an indel">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Raw read depth">
##INFO=<ID=RPB,Number=1,Type=Float,Description="Mann-Whitney U test of Read Position Bias (bigger is better)">
##INFO=<ID=MQB,Number=1,Type=Float,Description="Mann-Whitney U test of Mapping Quality Bias (bigger is better)">
##INFO=<ID=BQB,Number=1,Type=Float,Description="Mann-Whitney U test of Base Quality Bias (bigger is better)">
##INFO=<ID=MQSB,Number=1,Type=Float,Description="Mann-Whitney U test of Mapping Quality vs Strand Bias (bigger is better)">
##INFO=<ID=SGB,Number=1,Type=Float,Description="Segregation based metric.">
##INFO=<ID=MQ0F,Number=1,Type=Float,Description="Fraction of MQ0 reads (smaller is better)">
##INFO=<ID=ICB,Number=1,Type=Float,Description="Inbreeding Coefficient Binomial test (bigger is better)">
##INFO=<ID=HOB,Number=1,Type=Float,Description="Bias in the number of HOMs number (smaller is better)">
##INFO=<ID=DP4,Number=4,Type=Integer,Description="Number of high-quality ref-forward , ref-reverse, alt-forward and alt-reverse bases">
##INFO=<ID=MQ,Number=1,Type=Integer,Description="Average mapping quality">

## SNPs
We are going to use two different SNP VCFs, one has all the sites identified both by GIAB and us (`TP.chr20.vcf.gz`) and the other was identified by us and not by GIAB and are considered False Positive sites (`FP.chr20.vcf.gz`). Then we extract the annotations for each of the files by doing:

Now, we read-in the annotations in each of the files into a Pandas dataframe

In [8]:
import pandas as pd

%matplotlib inline

# read-in the data frame from tsv file
DF_TP=pd.read_csv('/Users/ernesto/SCRATCH/VARIANT_FILTERING/TP_annotations.tsv',sep="\t",
               names=['chr','pos','DP','RPB','MQB','BQB','MQSB','SGB','MQ0F','ICB','HOB',
                     'DP4','MQ'])
DF_FP=pd.read_csv('/Users/ernesto/SCRATCH/VARIANT_FILTERING/FP_annotations.tsv',sep="\t",
               names=['chr','pos','DP','RPB','MQB','BQB','MQSB','SGB','MQ0F','ICB','HOB',
                     'DP4','MQ'])

Now we will add a new column named `is_valid` to our two dataframes that will be 1 if the variant is real and will be 0 when the variant is a false positive. This new column will be the dependent binary variable in our classifier

In [16]:
DF_TP=DF_TP.assign(is_valid=1)
DF_FP=DF_FP.assign(is_valid=0)
frames = [DF_TP,DF_FP]
result = pd.concat(frames)

In [17]:
print(result)

        chr       pos     DP          RPB          MQB          BQB  \
0     chr20     68303  17127     0.794352            0  3.57131e-16   
1     chr20     72982  17678    0.0635767            0    0.0368648   
2     chr20     73765  17910  0.000533952            0            0   
3     chr20     77005  19513     0.911788            0  4.80673e-11   
4     chr20     78705  20994     0.890914            0    0.0353932   
5     chr20     80457  18725     0.994843            0     0.101913   
6     chr20     81154  22406     0.965516            0  1.04771e-14   
7     chr20     82603  18458      0.69988            0     0.763813   
8     chr20     83158  21736     0.754186            0            0   
9     chr20     85259  23468     0.162256            0            0   
10    chr20     85729  19689     0.757914            0  8.31719e-18   
11    chr20     88108  21684   0.00837126            0            0   
12    chr20     92251  15036     0.997864            0     0.606669   
13    